In [23]:
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter
from sklearn.linear_model import LogisticRegression
from sklearn import ensemble
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
%matplotlib inline

In [2]:
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text

In [3]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]

In [11]:
caesar = gutenberg.raw('shakespeare-caesar.txt')
hamlet = gutenberg.raw('shakespeare-hamlet.txt')

caesar = text_cleaner(caesar)
hamlet = text_cleaner(hamlet)

In [12]:
nlp = spacy.load("en")
caesar_doc = nlp(caesar[:5000])
hamlet_doc = nlp(hamlet[:5000])

In [13]:
caesar_sents = [[sent, "Caesar"] for sent in caesar_doc.sents]
hamlet_sents = [[sent, "Hamlet"] for sent in hamlet_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(caesar_sents + hamlet_sents)
sentences.head()

,0,1
0,"(Actus, Primus, .)",Caesar
1,"(Scoena, Prima, .)",Caesar
2,"(Enter, Flauius, ,, Murellus, ,, and, certaine...",Caesar
3,"(Flauius, .)",Caesar
4,"(Hence, :, home, you, idle, Creatures, ,, get,...",Caesar


In [14]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df

In [15]:
# Set up the bags.
caesarwords = bag_of_words(caesar_doc)
hamletwords = bag_of_words(hamlet_doc)

# Combine bags to create a set of unique words.
common_words = set(caesarwords + hamletwords)

In [16]:
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0


,implements,channell,shop,foode,blast,weepe,fear,parle,cicero,respect,...,knew,sensible,sight,come,course,have,certaine,watch,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Actus, Primus, .)",Caesar
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Scoena, Prima, .)",Caesar
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,"(Enter, Flauius, ,, Murellus, ,, and, certaine...",Caesar
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Flauius, .)",Caesar
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Hence, :, home, you, idle, Creatures, ,, get,...",Caesar


In [20]:
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)

In [21]:
lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(133, 602) (133,)
Training set score: 0.9774436090225563

Test set score: 0.7752808988764045


In [25]:
cross_val_score(lr, X_test, y_test, cv=5)

array([0.47368421, 0.88888889, 0.77777778, 0.64705882, 0.70588235])